# T5 VS Gemma-3-4B

## Setup & Model Loading

Environment configuration and GPU detection

In [ ]:
!pip install transformers accelerate sentencepiece torch --quiet
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Model initialization with memory tracking

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
import time
import pandas as pd
t5_name = "t5-small"
t5_tokenizer = AutoTokenizer.from_pretrained(t5_name)
t5_model = AutoModelForSeq2SeqLM.from_pretrained(t5_name).to(device)
gemma_name = "google/gemma-3-4b"

try:
    gemma_tokenizer = AutoTokenizer.from_pretrained(gemma_name)
    gemma_model = AutoModelForCausalLM.from_pretrained(
        gemma_name,
        torch_dtype=torch.bfloat16,
        device_map="auto")
except Exception as e:
    print("Could not load Gemma-3-4B:", e)
    gemma_tokenizer, gemma_model = None, None

prompts = [
    "Short lesson (500-1000 words)",
    "Medium lesson (2000-3000 words)",
    "Long lesson (5000+ words)"
]

results = []

Baseline memory usage measurement

In [ ]:
for prompt in prompts:
    row = {"prompt": prompt}
    t5_inputs = t5_tokenizer(prompt, return_tensors="pt").to(device)
    start = time.time()
    t5_outputs = t5_model.generate(**t5_inputs, max_new_tokens=64)
    end = time.time()
    row["t5_output"] = t5_tokenizer.decode(t5_outputs[0], skip_special_tokens=True)
    row["t5_runtime"] = end - start

    torch.cuda.reset_peak_memory_stats()
    start_mem = torch.cuda.memory_allocated()
    outputs = model.generate(**inputs, max_new_tokens=64)
    end_mem = torch.cuda.memory_allocated()
    peak_mem = torch.cuda.max_memory_allocated()

    if gemma_model:
        gemma_inputs = gemma_tokenizer(prompt, return_tensors="pt").to(device)
        start = time.time()
        gemma_outputs = gemma_model.generate(**gemma_inputs, max_new_tokens=64)
        end = time.time()
        row["gemma_output"] = gemma_tokenizer.decode(gemma_outputs[0], skip_special_tokens=True)
        row["gemma_runtime"] = end - start
    else:
        row["gemma_output"] = "<Gemma not loaded>"
        row["gemma_runtime"] = None
    
    results.append(row)

df = pd.DataFrame(results)
df

## Performance Benchmarking

Memory usage comparison charts

Inference speed analysis

In [ ]:
plt.figure(figsize=(8,5))
plt.bar(df.index - 0.15, df["t5_runtime"], width=0.3, label="T5")
plt.bar(df.index + 0.15, df["gemma_runtime"], width=0.3, label="Gemma")
plt.xticks(df.index, [f"Prompt {i+1}" for i in df.index])
plt.ylabel("Runtime (s)")
plt.title("Inference Speed")
plt.legend()
plt.show()

Output length and generation time

In [ ]:
plt.figure(figsize=(8,5))
plt.scatter(df["t5_tokens"], df["t5_runtime"], label="T5", marker="o")
plt.scatter(df["gemma_tokens"], df["gemma_runtime"], label="Gemma", marker="x")
plt.xlabel("Output tokens")
plt.ylabel("Runtime (s)")
plt.title("Output Length vs Generation Time")
plt.legend()
plt.show()

if torch.cuda.is_available():
    plt.figure(figsize=(8,5))
    plt.bar(df.index - 0.15, [m/1e6 for m in df["t5_mem_peak"]], width=0.3, label="T5")
    plt.bar(df.index + 0.15, [m/1e6 for m in df["gemma_mem_peak"]], width=0.3, label="Gemma")
    plt.xticks(df.index, [f"Prompt {i+1}" for i in df.index])
    plt.ylabel("Peak Memory (MB)")
    plt.title("Memory Usage")
    plt.legend()
    plt.show()

Batch processing performace

In [ ]:
batch_prompts = ["Hello world!" for _ in range(4)]
batch_sizes = [1, 2, 4]
batch_results = []

for bs in batch_sizes:
    inputs = t5_tokenizer(batch_prompts[:bs], return_tensors="pt", padding=True).to(device)
    start = time.time()
    _ = t5_model.generate(**inputs, max_new_tokens=32)
    t5_time = time.time() - start

    if gemma_model:
        inputs = gemma_tokenizer(batch_prompts[:bs], return_tensors="pt", padding=True).to(device)
        start = time.time()
        _ = gemma_model.generate(**inputs, max_new_tokens=32)
        gemma_time = time.time() - start
    else:
        gemma_time = None

    batch_results.append({"batch_size": bs, "t5_time": t5_time, "gemma_time": gemma_time})

batch_df = pd.DataFrame(batch_results)
display(batch_df)

plt.figure(figsize=(8,5))
plt.plot(batch_df["batch_size"], batch_df["t5_time"], label="T5", marker="o")
if gemma_model:
    plt.plot(batch_df["batch_size"], batch_df["gemma_time"], label="Gemma", marker="x")
plt.xlabel("Batch size")
plt.ylabel("Runtime (s)")
plt.title("Batch Processing Performance")
plt.legend()
plt.show()

## Qualitative Output Assessment

Side-by-side summary comparison

In [ ]:
summary_cols = [
    "t5_runtime", "gemma_runtime",
    "t5_tokens", "gemma_tokens",
    "t5_mem_peak", "gemma_mem_peak"
]

summary_df = df[["prompt"] + summary_cols]
summary_df

comparison = pd.concat([
    df[["prompt", "t5_runtime", "t5_tokens", "t5_mem_peak"]].rename(
        columns={"t5_runtime": "runtime", "t5_tokens": "tokens", "t5_mem_peak": "peak_mem"}
    ).assign(model="T5"),
    
    df[["prompt", "gemma_runtime", "gemma_tokens", "gemma_mem_peak"]].rename(
        columns={"gemma_runtime": "runtime", "gemma_tokens": "tokens", "gemma_mem_peak": "peak_mem"}
    ).assign(model="Gemma")
])

comparison = comparison.pivot(index="prompt", columns="model", values=["runtime", "tokens", "peak_mem"])
comparison

## Practical Recomendations

Hardware requirement guidelines

- **T5 (small/base/large)**
  - Memory footprint: 220M – 770M parameters.
  - Runs comfortably on CPU, GPU optional for speedup.
  - Peak GPU memory: ~1–2 GB for `t5-small`, ~4–6 GB for `t5-large`.
  - Faster inference on modest hardware, but lower output fluency.

- **Gemma-3-4B**
  - 4B parameters → heavier model.
  - GPU strongly recommended: needs ~8–12 GB VRAM for inference.
  - CPU-only is possible but very slow.
  - Better fluency, coherence, and longer-context handling, but higher hardware cost.

Performance vs quality trade-offs

- **T5**
  - Pros: Fast, lightweight, lower memory usage, good for translation and summarization tasks.
  - Cons: Output may be shorter, less nuanced, and sometimes rigid compared to larger models.

- **Gemma-3-4B**
  - Pros: Generates richer, more natural text; handles open-ended prompts better.
  - Cons: Slower inference, higher VRAM usage, may require batching limits.

Choose T5 for speed and low resources, and Gemma for quality and richer outputs if you have the hardware budget.